# Worked Example on Solar Cells

## Introduction

Organic photovoltaic (OPV) are ‘plastic’ solar cells that can be made cheaply and easily as you can use techniques like roll to roll printing, inject printing and spray coating. Current generation solar cells take several years of use before they payback the energy required in their manufacture, OPVs are so efficient that their energy payback is only 24hours. Power conversion efficiencies (PCEs) of OPVs are now around 14%. To commercialise them, we need to figure out how best to manufacture them.

Organic photovoltaic devices have a sandwich architecture. The bottom layers Al/Mg and LiF are the bottom electrode. The important part is the bulk hetereojunction, shown in red in the figure below, which comprises of a low band gap polymer which is the electron donor and fullerene which is the electron acceptor. Addition of an additive helps with forming and bridging separate nanodomains of donor and accceptor. Solar cells work by using light to form an exciton which then separates into an electron-hole pair and you want these to be separated from each other, which is why you want separate nanodomain of donor and acceptor. The top of the solar cell is PEDOT:PSS (a conducting polymer) and ITO  (indium tin oxide), a see-through electrode, which together act as the top electrode. 

![Test_HetJunc.jpeg](Test_HetJunc.jpeg)

Figure 1. (a) Schematic of single junction organic photovoltaic (OPV) devices, showing the bulk heterojunction (BHJ; in red), and the multiple interfacial layers in the device. (b) Schematic of BHJ morphology: in this case, a low band gap polymer donor and a fullerene acceptor undergoing nanoscale phase segregation into discrete nanoscale domains of donor and acceptor. The use of an additive is often purported to assist in nanodomain formation, as shown here. Taken from [ACS Nano 2018, 12, 7434−7444]





## The task

The task is to optimise the construction of this type of solar cell. Donor weight percentage is a measure of the ratio of donor to acceptor in the heterojunction.  Total solution concentration is the concentration of the spin-coating solution. Bulk heterojunction spin-case speed is a measure of how fast you spin the device when coating it with the bulk heterojunction mixture. Processing additive is the amount of additive (diiodooctane) added to the mixture. The thickness of a spun film is determined by the spin speed, solvent vapour pressure and solution viscosity, as both the donor weight percentage and total solution concentration can affect viscosity, the first three factors can all affect the thickness of the final BHJ layer. The additive (diiodooctane) increases the drying time for the film, helping to separate the hetereojunction out into nanodomains of donor and acceptor rich areas. 


**Factors selected:**

| Name  |   Factors                         |Factor range   | No. of levels  |
|-------|-----------------------------------|---------------|----------------|
| Donor | Donor weight percentage          | 10-55 (wt %)         | 4       |
| Conc. | Total solution concentration     | 10-25 (mg/mL)        | 4       |
| Spin  | Bulk heterojunction spin-case speed | 600 - 3000  (rpm) | 4       |
| Add.  | Processing additive              |  0-12 (vol %)        | 4       | 

We shall use the shortened names from the table above. 

**Files: **

1. `solar_cells_1.csv` results from the first experiment, fractional factorial, 4 factors and 4 levels, here we have 16 experiments, one failed to solidify. 
2. `'solar_cells_2.csv'` has results from the second experiment, a fractional factorial, 3 factors and 3 levels. This covers a smaller range. 

The data is taken from: 
"How To Optimize Materials and Devices via Design of Experiments and Machine Learning: Demonstration Using Organic Photovoltaics", Bing Cao, Lawrence A. Adutwum, Anton O. Oliynyk, Erik J. Luber, Brian C. Olsen, Arthur Mar, and Jillian M. Buriak, ACS Nano 2018, 12, 7434−7444

### Importing packages

In [ ]:
# for dataframes
import pandas as pd

# for pictures
import matplotlib.pyplot as plt

# for maths
import numpy as np

import doenut

# set the log level
import logging
doenut.set_log_level(logging.WARNING)

### Read in the first experiment's data

In [ ]:
df = pd.read_csv("data/solar_cells_1.csv")
df

### Set up input and response dataframes

We must drop the last experiment, as these devices didn't set.

In [ ]:
inputs = pd.DataFrame(
    {
        "Donor %": [float(x) for x in df.iloc[1:-1, 1]],
        "Conc.": [float(x) for x in df.iloc[1:-1, 2]],
        "Spin": [float(x) for x in df.iloc[1:-1, 3]],
        "Add.": [float(x) for x in df.iloc[1:-1, 4]],
    }
)
inputs

In [ ]:
responses = pd.DataFrame({"PCE": [float(x) for x in df["PCE"][1:-1]]})
responses

## Task 1. Create a linear (main factors only) model

Create a linear model, i.e. a model that has just the main effects (also known as a first order model or main effects model) Fit your linear model to the first experiment’s data and calculate R2 and Q2 for your fitted model. Then answer the questions.

In [ ]:
# First create a dataset object linking the inputs and responses.
data_set = doenut.data.ModifiableDataSet(inputs, responses)

model = doenut.models.AveragedModel(data_set)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")

## Task 2. Create a quadratic, parsimonious and hierarchical model

Create a **quadratic**, **parsimonious** and **hierarchical** model. Starting with a quadratic model, and making sure that all models are hierarchical, optimise the model by removing **only** the **statistically insignificant** terms. Keep a note of the terms removed and teh $R^2$ and $Q^2$ values.

First we must expand the input dataframe to include the higher order terms.

In [ ]:
sat_source_list = []
source_list = []
sat_inputs_orig, sat_source_list = doenut.add_higher_order_terms(
    inputs, add_squares=True, add_interactions=True, column_list=[]
)

### Fully saturated quadratic model:

This contains all the main terms and all the square terms.

In [ ]:
# This time we only want to select some columns and we want it to scale each column so the values are normalised
# across columns.
# First make a list of the columns we want
input_selector = [
    "Donor %",
    "Conc.",
    "Spin",
    "Add.",
    "Donor %**2",
    "Conc.**2",
    "Spin**2",
    "Add.**2",
]
# Note we could also use list indices like this:
# input_selector = [0, 1, 2, 3, 4, 5, 6, 7]
dataset = (
    doenut.data.ModifiableDataSet(sat_inputs_orig, responses)
    .filter(input_selector)
    .scale()
)

model = doenut.models.AveragedModel(
    dataset, scale_run_data=True, drop_duplicates="no"
)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")
doenut.plot.plot_summary_of_fit_small(r2, q2)
doenut.plot.coeff_plot(
    model.coeffs,
    labels=list(dataset.get().inputs.columns),
    errors="p95",
    normalise=True,
)

In [ ]:
# That's not very good. Let's try with a different label selection, removing the most obviously insignificant terms
input_selector = [0, 1, 2, 4, 5, 6]
dataset = (
    doenut.data.ModifiableDataSet(sat_inputs_orig, responses)
    .filter(input_selector)
    .scale()
)

model = doenut.models.AveragedModel(
    dataset, scale_run_data=True, drop_duplicates="no"
)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")
doenut.plot.plot_summary_of_fit_small(r2, q2)
doenut.plot.coeff_plot(
    model.coeffs,
    labels=list(dataset.get().inputs.columns),
    errors="p95",
    normalise=True,
)

15 datapoints so 14 DoF.

Starting from quadratic model

| No. of terms | DoF   | term removed | factor removed | $R^2$  |  $Q^2$ |
|--------------|-------|--------------|----------------|--------|--------|
| 8            | 6     |              |                | 0.815  | -0.176 |
| 7            | 7     | 8            | `'Add.**2`     | 0.813  | 0.0863 |
| 6            | 8     | 3            | `Add.`         | 0.813  | 0.332  |

this is the model with no statistically insignificant terms. It's heirarchical. 

The Q2 is better than the main effects only model

## Task 3. Create a parsimonious interaction model

Create hierarchical parsimonious interaction model. Starting with a interaction model, and making sure that all models are hierarchical, optimise the model by removing only the statistically insignificant terms. Keep a note of the terms removed and the $Q^2$ and $R^2$ values.


In [ ]:
input_selector = [0, 1, 2, 3, 9, 10]

dataset = (
    doenut.data.ModifiableDataSet(sat_inputs_orig, responses)
    .filter(input_selector)
    .scale()
)

model = doenut.models.AveragedModel(
    dataset, scale_run_data=True, drop_duplicates="no"
)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")
doenut.plot.plot_summary_of_fit_small(r2, q2)
doenut.plot.coeff_plot(
    model.coeffs,
    labels=list(dataset.get().inputs.columns),
    errors="p95",
    normalise=True,
)


15 terms so 14 DoF

Starting from square model

| No. of terms | DoF   | term removed | factor removed | $R^2$  |  $Q^2$ |
|--------------|-------|--------------|----------------|--------|--------|
| 10           | 4     |              |                | 0.811  |  -1.79 |
| 9            | 5     |      12      | `Conc.*Add.`   | 0.813  | 0.0863 |
| 8            | 6     |      13      | `'Spin*Add.'`  | 0.798  | -0.555 |
| 7            | 7     |      11      | `Conc*Spin.`   | 0.777  | 0.0133 |
| 6            | 8     |      8       | `Donor*Conc.`  | 0.760  | 0.310  | 

this is the model with no statistically insignificant terms. It's heirarchical. 

The Q2 is better than the main effects only model, but not as good as the square terms. 

## Task 4: Combine data from both experiments and train a parsimonious model
### Loading the second dataset

In [ ]:
df = pd.read_csv("data/solar_cells_2.csv")
df

In [ ]:
inputs_2 = pd.DataFrame(
    {
        "Donor %": [float(x) for x in df.iloc[1:-1, 1]],
        "Conc.": [float(x) for x in df.iloc[1:-1, 2]],
        "Spin": [float(x) for x in df.iloc[1:-1, 3]],
    }
)
inputs_2

In [ ]:
responses_2 = pd.DataFrame({"PCE": [float(x) for x in df["PCE"][1:-1]]})
responses_2

In [ ]:
inputs[["Donor %", "Conc.", "Spin"]]

### Building a combined dataset

In [ ]:
new_inputs = pd.concat(
    [inputs[["Donor %", "Conc.", "Spin"]], inputs_2], axis=0
)
new_responses = pd.concat([responses, responses_2], axis=0)

In [ ]:
new_inputs

In [ ]:
sat_source_list = []
source_list = []
sat_inputs_2, sat_source_list = doenut.add_higher_order_terms(
    new_inputs, add_squares=True, add_interactions=True, column_list=[]
)

### Saturated model: 9 terms

In [ ]:
input_selector = [0, 1, 2, 3, 4, 5, 6, 7, 8]

dataset = doenut.data.ModifiableDataSet(sat_inputs_2, new_responses).filter(
    input_selector
)

model = doenut.models.AveragedModel(
    dataset, scale_data=True, drop_duplicates="no"
)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")
doenut.plot.plot_summary_of_fit_small(r2, q2)
doenut.plot.coeff_plot(
    model.coeffs,
    labels=list(dataset.get().inputs.columns),
    errors="p95",
    normalise=True,
)

### Optimised parsimonious model

In [ ]:
input_selector = [0, 1, 2, 3, 4, 5]

dataset = doenut.data.ModifiableDataSet(sat_inputs_2, new_responses).filter(
    input_selector
)

model = doenut.models.AveragedModel(
    dataset, scale_data=True, drop_duplicates="no"
)

r2, q2 = model.r2, model.q2

print(f"R2 is {r2}, Q2 is {q2}")
doenut.plot.plot_summary_of_fit_small(r2, q2)
doenut.plot.coeff_plot(
    model.coeffs,
    labels=list(dataset.get().inputs.columns),
    errors="p95",
    normalise=True,
)

In [ ]:
27 - 1 - 9


| No. of terms | DoF   | term removed | factor removed | $R^2$  |  $Q^2$ |
|--------------|-------|--------------|----------------|--------|--------|
| 9            |   17  |              |                | 0.89   | -0.425 |
| 8            |   18  |    8         | `Conc.*Spin`   | 0.887. | 0.44   |
| 7            |   19  |    7         | `Donor*Spin`   | 0.88   | 0.535  |
| 6            |   20  |    6         | `Donor* Conc`  | 0.871  | 0.695  | 

## Task 5: Optimising the devices. Using the best model that you have trained (as measured by Q2), find some conditions to optimise the devices. 

### Method 1: Plot a 4-D contour plot and read the values off:

In [ ]:
#'Donor %', 'Conc.', 'Spin'

n_points = 60


def my_function(df_1):
    ## Put the two main factors that you're not plotting here
    ## set them to sensible constant values

    df_1["Donor %**2"] = df_1["Donor %"] * df_1["Donor %"]
    df_1["Conc.**2"] = df_1["Conc."] * df_1["Conc."]
    df_1["Spin**2"] = df_1["Spin"] * df_1["Spin"]

    return df_1


c_key = "Spin"
y_key = "Conc."
x_key = "Donor %"

doenut.plot.four_D_contour_plot(
    unscaled_model=model.model,
    x_key=x_key,
    y_key=y_key,
    c_key=c_key,
    x_limits=[inputs[x_key].min(), inputs[x_key].max()],
    y_limits=[inputs[y_key].min(), inputs[y_key].max()],
    constants=[500, 1500, 2500],
    n_points=60,
    my_function=my_function,
    input_selector=[],
    fig_label="Solar Cells",
    x_label=x_key,
    y_label=y_key,
    constant_label=c_key,
    z_label="PCE",
    cmap="jet",
    num_of_z_levels=16,
    z_limits=[0, 12],
)

### Method 2. Run the model on the input values: 

In [ ]:
question_5p2 = pd.DataFrame(
    {
        "A": {"Donor %": 20, "Conc.": 12, "Spin": 500},
        "B": {"Donor %": 40, "Conc.": 16, "Spin": 1500},
        "C": {"Donor %": 35, "Conc.": 22, "Spin": 1500},
        "D": {"Donor %": 45, "Conc.": 18, "Spin": 2500},
        "E": {"Donor %": 20, "Conc.": 17, "Spin": 2500},
    }
).T

In [ ]:
question_5p2

In [ ]:
question_5p2.index

In [ ]:
sat_source_list = []
source_list = []
sat_inputs_q5, sat_source_list = doenut.add_higher_order_terms(
    question_5p2,
    add_squares=True,
    add_interactions=True,
    column_list=[],
)

results, _ = doenut.predict_from_model(
    model.model, sat_inputs_q5, input_selector
)
letters = [x for x in question_5p2.index]
[print(f"{letters[i]}:\t{results[i]}") for i in range(len(letters))];

Answer C is above 7.